In [1]:
import requests
import base64
import json
import re
import hashlib

# import mysql.connector
from dotenv import load_dotenv
import os


In [2]:
def campos(fields):
    cad = ''
    for x in fields.keys():
        cad += x + ' = ' + fields[x] + ', '
    return cad[:-2]

def leeactzl(user, repo_name, path_to_file):
    json_url ='https://api.github.com/repos/{}/{}/contents/{}'.format(user, repo_name,
                                                                      path_to_file)
    response = requests.get(json_url) #get data from json file located at specified URL 

    if response.status_code == requests.codes.ok:
        jsonResponse = response.json()  # the response is a JSON
        #the JSON is encoded in base 64, hence decode it
        content = base64.b64decode(jsonResponse['content'])
        #convert the byte stream to string
        jsonString = content.decode('utf-8')
        return json.loads(jsonString)
 
    else:
        return 'Content was not found.'
    
def calcquerys(finalJson):
    
    querys = []
    for registro in finalJson:
        #if registro['firma'] == hashlib.sha1(registro["instruccion"].encode('utf-8')).hexdigest():
        dt_query = json.loads(registro['instruccion'])

        quetmp = []
        for y in dt_query.keys():
            if dt_query[y]['op'] == 'UPDATE':
                query  = str(dt_query[y]['op']) + ' ' + y + ' SET ' + campos(dt_query[y]['fields'])
                query += ' WHERE '
            elif dt_query[y]['op'] == 'INSERT INTO':
                query  = query = str(dt_query[y]['op']) + ' ' + y
                query += ' ' + str(tuple(list(dt_query[y]['fields'].keys())))+ ' VALUES '
                query +=  str(tuple(list(dt_query[y]['fields'].values())))
                query  = query.replace("'", "")
                filtro_update = 'coduser = ' + dt_query[y]['fields']['coduser']
            quetmp.append(query)

        quetmp = [q + filtro_update if 'UPDATE' in q else q for q in quetmp]

        querys.extend(quetmp)

            #break # <--== Pilas quitar en produccion

    
    patron = re.compile('[0-9]{4}-[0-9]{2}-[0-9]{2}')

    for i, query in enumerate(querys):
        for r in patron.findall(querys[i]):
            querys[i] = querys[i].replace(r, "'" + r + "'")

    return querys

def conexion(query):
    try:
        cnx = mysql.connector.connect(user=sql_id, password=sql_pw, host='127.0.0.1', database='clientes')
        cursor = cnx.cursor()
        seleccion = (query)
        cursor.execute(seleccion)
        cnx.commit()
        return 'ok'
    except:
        cnx = mysql.connector.connect(user=sql_id, password=sql_pw, host='127.0.0.1', database='clientes')
        cursor = cnx.cursor()
        seleccion = (query)
        cursor.execute(seleccion)
        return list(cursor)

In [28]:
user = 'sistelca'
repo_name = 'desechosSolidos'
path_to_file = 'data_actzl.json'

finalJson = leeactzl(user, repo_name, path_to_file)


In [34]:
instrucions =  {'update': ['set', 'where'],
                 'insert into': ['(', ') values'],
                 'delete from': ['where', 'todo']
               } 

for registro in finalJson:
    #if registro['firma'] == hashlib.sha1(registro["instruccion"].encode('utf-8')).hexdigest():
    dt_query = json.loads(registro['instruccion'])
    regq = ''
    for y in dt_query.keys():
        z = dt_query[y]
        asi = z['op'].lower()
        gna = instrucions[asi][0]
        fl  = instrucions[asi][1]
        regq += z['op'] +' ' + y + ' ' + gna + z['set'] + fl + z['filtro']
        print(z['op'] +' ' + y + ' ' + gna + z['set'] + fl + z['filtro'])
#     print(regq)
    print(registro['firma'] == hashlib.sha1(regq.lower().encode('utf-8')).hexdigest())
    


UPDATE datos_red set fech_pag='2021-05-09' where coduser='321'
INSERT INTO histori_pags (coduser, fech_pag, cant_Bf, fech_venc, anfit, cod) values (321, '2021-05-09', 200000, '2021-06-09', '192.168.45.224', '')
UPDATE datos_per set fech_ing='2021-05-09', fech_ven='2021-06-09' where coduser='321'
False
UPDATE datos_red set fech_pag='2021-05-10' where coduser='313'
INSERT INTO histori_pags (coduser, fech_pag, cant_Bf, fech_venc, anfit, cod) values (313, '2021-05-09', 200000, '2021-06-10', '192.168.45.222', '211510b0154555d6cc9ff91d9fcbf1b1')
UPDATE datos_per set fech_ing='2021-05-09', fech_ven='2021-06-10' where coduser='313'
False
UPDATE datos_red set fech_pag='2021-08-01' where coduser='341'
INSERT INTO histori_pags (coduser, fech_pag, cant_Bf, fech_venc, anfit, cod) values (341, '2021-05-10', 100000, '2021-09-01', '192.168.45.222', '962ad89f380951874518b9f027ef2a6f')
UPDATE datos_per set fech_ing='2021-05-10', fech_ven='2021-09-01' where coduser='341'
True
